<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

FEARFRED's generator creates and samples from a flow field. The Flashlight Kernel converts these samples into a directed graph. At last, we arrive at the final piece in our architecture: a GNN to discriminate between graphs coming from our dataset and graphs coming from the flow field. 

The trick here is to preserve the chain of differentiability between the generator and discriminator. This makes any translation of the affinity matrix from the flashlight kernel into a PyTorch Geometric format problematic -- this translation isn't differentiable! It's better to stay as close to the affinity matrix as possible.

To do this, we adopt a *learnable scattering network*, LEGSNet, which operates entirely on the affinity matrix of a graph. This instatiation of LEGSNet uses the scattering transform as our *witness function*, paired with a simple feedforward classifier to make the final call on in-group membership.

# Fixed LEGS Module
Adapted from Alex Tong's LEGSNet, this module implements a vanilla scattering network on an adjacency matrix and degree information. In this simpler discriminator, there are no learnable parameters in the scattering transform -- only in the classifier.

In [1]:
#|output: asis
#| echo: false
show_doc(scatter_moments)

---

[source](https://github.com/professorwug/FEARFRED/blob/main/FEARFRED/discriminator.py#L7){target="_blank" style="float:right; font-size:smaller"}

### scatter_moments

>      scatter_moments (data, moments_returned=2)

In [ ]:
data = torch.rand(5,3)
print("data is ",data)
scatter_moments(data,moments_returned=4)

data is  tensor([[0.1032, 0.8670, 0.3709],
        [0.0773, 0.7655, 0.6835],
        [0.5227, 0.0349, 0.0360],
        [0.7974, 0.5565, 0.8710],
        [0.0891, 0.4023, 0.8274]])


tensor([[0.4470, 0.5088, 0.1979, 0.7416, 0.4396, 0.1001, 0.0942, 0.0528, 0.0180,
         0.0915]])

In [2]:
#|output: asis
#| echo: false
show_doc(ScatteringDiscriminator)

---

[source](https://github.com/professorwug/FEARFRED/blob/main/FEARFRED/discriminator.py#L64){target="_blank" style="float:right; font-size:smaller"}

### ScatteringDiscriminator

>      ScatteringDiscriminator (n_nodes, n_features)

Given an affinity matrix and associated node features, classifies: 
does this belong to the true dataset?
The number of nodes per graph must be constant.

|    | **Details** |
| -- | ----------- |
| n_nodes | number of nodes in inputted graphs |
| n_features | number of features per node |

## Testing

In [ ]:
SD = ScatteringDiscriminator(n_nodes=3,n_features=2)

In [ ]:
A = torch.rand(3,3)
x = torch.ones(3,2)
SD(A,x)

tensor([[-0.0809]], grad_fn=<AddmmBackward0>)